# Read Data

In [1]:
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
import missingno as msno

# new versions


# March 27
# T2 = pd.read_csv('StataReg/CountyVaccine-03-27.csv')


# April 07
# T2 = pd.read_csv('StataReg/CountyVaccine-04-07.csv')


# April 19
# T2 = pd.read_csv('StataReg/CountyVaccine-04-19.csv') 
# T2 = pd.read_csv('StataReg/CountyVaccine-04-19-reviewversion-v4.csv') # with ZWhiteRate

# May 20
T2 = pd.read_csv('StataReg/CountyVaccine-05-20.csv') 
# T2 = pd.read_csv('StataReg/CountyVaccine-05-20-reviewversion-v2.csv')

In [2]:
# calculate the ZWhiteNum
# ZWhiteNum just the combination of CvdVax_NHWhiteNum and CvdVax_WhiteNum.

# for LeastOne Vaccination
for col in ['CvdVax_WhiteNum', 'CvdVax_NHWhiteNum']:
    index = - T2[col].isna()
    T2.loc[index, 'CvdVax_ZWhiteNum'] = T2.loc[index, col] 

# for Full Vaccination  
try:
    for col in ['FullCvdVax_WhiteNum', 'FullCvdVax_NHWhiteNum']:
        index = - T2[col].isna()
        T2.loc[index, 'FullCvdVax_ZWhiteNum'] = T2.loc[index, col] 
except:
    print('No Full Data')

## State ReportNH



In [3]:
ReportNH = ['Illinois', 'Texas', 'Virginia', 'California', 'New Jersey',  'Oregon']

In [4]:
T2[[i for i in T2.columns if 'Rate' in i]].describe().T

,count,mean,std,min,25%,50%,75%,max
CvdVax_ZWhiteRate,1175.0,35.632056,16.671715,1.697167,24.901830,33.259083,44.825970,86.300399
CvdVax_WhiteRate,632.0,33.651355,16.385307,1.602367,25.252131,34.164368,42.950965,75.757706
CvdVax_NHWhiteRate,543.0,35.971036,15.848531,6.674082,23.949654,31.206404,44.331697,76.999898
CvdVax_MWhiteRate,1175.0,34.723344,16.174062,1.602367,24.637795,32.708522,43.014095,76.999898
CvdVax_MBlackRate,1175.0,21.510127,11.522223,0.000000,13.908919,20.544203,27.333525,77.944862
...,...,...,...,...,...,...,...,...
A20T74_BlackRate,1175.0,67.762479,8.087258,35.714286,64.027364,67.260621,70.326576,96.228110
A15T74_WholeRate,1175.0,74.083869,2.310578,65.922204,72.838772,74.112571,75.289000,85.659488
A15T74_WhiteRate,1175.0,74.276687,2.342991,65.808571,73.016070,74.391575,75.486160,85.912343
A15T74_NHWhiteRate,1175.0,74.795202,2.419403,58.243451,73.647823,74.785245,75.927535,86.054154


## Absolute Disparity (Disparity)

In [5]:
suffix = '_Disparity'

new_cols = []
for i in ['CvdVax', 'CvdVaxAbove15', 'CvdVaxAbove20']:
    new_col = i+suffix
    T2[new_col] = T2[ i + '_MWhiteRate'] - T2[i+'_MBlackRate'] 
    new_cols.append(new_col)
    

for i in  ['FluVax', 'HighSchool', 'Bachelor', 'IT', 'Unempl', 'A15T74', 'A20T74',  'Above75', 'Above65', 'Above15', 'Above20',]:
    new_col = i+suffix
    T2[new_col] = T2[ i + '_NHWhiteRate'] - T2[i+'_BlackRate'] 
    new_cols.append(new_col)

    
for i in [ 'MeanInc', 'MedianInc']:
    new_col = i+suffix
    T2[new_col] = T2[ i + '_NHWhiteAvg'] - T2[i+'_BlackAvg'] 
    new_cols.append(new_col)
    

try:
    for i in ['FullCvdVax', 'FullCvdVaxAbove15', 'FullCvdVaxAbove20']:
        new_col = i+suffix
        T2[new_col] = T2[ i + '_MWhiteRate'] - T2[i+'_MBlackRate'] 
        new_cols.append(new_col)
except:
    print('No Full Cvd')
    
    
# For ZCvdVax
# ZWhiteRate - MBlackRate
try:   
    for i in ['CvdVax', 'CvdVaxAbove15', 'CvdVaxAbove20']:
        new_col ='Z' + i + suffix
        T2[new_col] = T2[ i + '_ZWhiteRate'] - T2[i+'_MBlackRate'] 
        new_cols.append(new_col)

except:
    print('No ZCvd Info')

In [6]:
T2[new_cols].describe()

,CvdVax_Disparity,CvdVaxAbove15_Disparity,CvdVaxAbove20_Disparity,FluVax_Disparity,HighSchool_Disparity,Bachelor_Disparity,IT_Disparity,Unempl_Disparity,A15T74_Disparity,A20T74_Disparity,...,Above15_Disparity,Above20_Disparity,MeanInc_Disparity,MedianInc_Disparity,FullCvdVax_Disparity,FullCvdVaxAbove15_Disparity,FullCvdVaxAbove20_Disparity,ZCvdVax_Disparity,ZCvdVaxAbove15_Disparity,ZCvdVaxAbove20_Disparity
count,1175.000000,1175.000000,1175.000000,951.000000,1165.000000,1165.000000,1140.000000,1125.000000,1175.000000,1175.000000,...,1175.000000,1175.000000,1119.000000,824.000000,898.000000,898.000000,898.000000,1175.000000,1175.000000,1175.000000
mean,13.213217,14.225457,14.142996,14.503680,8.074421,9.482403,6.592895,-5.593956,-0.928942,1.396610,...,4.410747,6.736299,13349.390527,19123.583738,11.777884,13.010808,13.188835,14.121929,15.158009,15.086473
std,15.330566,18.169931,19.302720,8.160478,14.133217,13.749434,17.743925,12.675899,6.516359,7.735750,...,6.527773,8.066036,9417.236816,18488.456088,9.242346,10.649711,11.254987,15.662021,18.489460,19.623293
min,-62.344213,-79.113710,-85.464888,-22.000000,-36.900000,-86.600000,-49.800000,-98.400000,-24.481251,-26.597654,...,-17.636986,-21.917808,-59205.000000,-190604.000000,-19.603544,-23.042097,-24.188819,-61.725336,-78.491250,-84.846345
25%,4.019782,3.671941,3.056032,10.000000,0.600000,4.900000,-3.900000,-7.600000,-2.743063,-0.991405,...,1.101281,2.822501,8502.000000,11887.750000,5.037003,5.637832,5.308201,4.666418,4.376110,3.870575
50%,12.014652,12.773174,12.563308,15.000000,6.400000,10.900000,6.200000,-3.300000,-0.075721,2.038686,...,4.532884,6.684352,13112.000000,20650.500000,11.621961,12.520264,12.386066,12.792324,13.684091,13.153236
75%,21.702274,23.565453,24.005852,19.000000,12.700000,16.700000,14.500000,1.500000,2.258810,4.969221,...,7.485500,10.764705,18083.000000,28583.000000,17.735853,20.048050,20.760706,23.433264,25.225953,25.332354
max,72.782585,82.243386,85.610804,51.000000,96.300000,55.200000,83.200000,11.500000,26.163522,34.308096,...,43.563941,48.343816,87329.000000,112332.000000,59.964413,66.783069,69.600982,72.782585,82.243386,85.610804


## Ratio Disparity (RD)

In [7]:
new_cols = []

suffix = '_RD'

for i in ['CvdVax', 'CvdVaxAbove15', 'CvdVaxAbove20']:
    new_col = i+suffix
    T2[new_col] = (T2[ i + '_MWhiteRate'] + 0.0001) /( T2[i+'_MBlackRate']  + 0.0001)
    new_cols.append(new_col)
    

for i in  ['FluVax', 'HighSchool', 'Bachelor', 'IT', 'Unempl', 'A15T74', 'A20T74',  'Above75', 'Above65', 'Above15', 'Above20',]:
    new_col = i+suffix
    T2[new_col] = (T2[ i + '_NHWhiteRate']+ 0.0001) / (T2[i+'_BlackRate']  + 0.0001)
    new_cols.append(new_col)

    
for i in [ 'MeanInc', 'MedianInc']:
    new_col = i+suffix
    T2[new_col] = (T2[ i + '_NHWhiteAvg'] + 0.0001)/ (T2[i+'_BlackAvg']  + 0.0001)
    new_cols.append(new_col)
    

try:
    for i in ['FullCvdVax', 'FullCvdVaxAbove15', 'FullCvdVaxAbove20']:
        new_col = i+suffix
        T2[new_col] = (T2[ i + '_MWhiteRate'] + 0.0001)/( T2[i+'_MBlackRate']  + 0.0001)
        new_cols.append(new_col)
except:
    print('No Full Cvd')
    
    
try:   
    for i in ['CvdVax', 'CvdVaxAbove15', 'CvdVaxAbove20']:
        new_col ='Z' + i + suffix
        T2[new_col] = (T2[ i + '_ZWhiteRate'] + 0.0001) /( T2[i+'_MBlackRate']  + 0.0001)
        new_cols.append(new_col)

except:
    print('No ZCvd Info')
    
T2[new_cols].describe()

,CvdVax_RD,CvdVaxAbove15_RD,CvdVaxAbove20_RD,FluVax_RD,HighSchool_RD,Bachelor_RD,IT_RD,Unempl_RD,A15T74_RD,A20T74_RD,...,Above15_RD,Above20_RD,MeanInc_RD,MedianInc_RD,FullCvdVax_RD,FullCvdVaxAbove15_RD,FullCvdVaxAbove20_RD,ZCvdVax_RD,ZCvdVaxAbove15_RD,ZCvdVaxAbove20_RD
count,1175.000000,1175.000000,1175.000000,951.000000,1165.000000,1165.000000,1140.000000,1125.000000,1175.000000,1175.000000,...,1175.000000,1175.000000,1119.000000,824.000000,898.000000,898.000000,898.000000,1175.000000,1175.000000,1175.000000
mean,527.559157,585.620096,605.839928,9065.591571,6194.263737,23387.107710,8746.792427,8624.795255,0.994814,1.033958,...,1.062247,1.107706,2.392742,1.712752,1952.192997,2260.291541,2388.462226,559.335355,604.717596,617.200115
std,18012.478204,20005.960289,20701.467061,57061.917932,74600.814300,67312.670343,81554.393596,19566.292245,0.084800,0.121852,...,0.092209,0.136793,4.078078,1.421653,25574.497990,29106.088454,30516.424729,19100.024514,20659.215900,21089.618998
min,0.121444,0.115820,0.114014,0.620001,0.604502,0.134001,0.502000,0.010002,0.752598,0.722634,...,0.823630,0.780822,0.342218,0.237584,0.418982,0.418884,0.426189,0.130165,0.122777,0.120426
25%,1.207267,1.159853,1.132885,1.278423,1.006296,1.292681,0.955495,0.369461,0.964473,0.985865,...,1.013187,1.037356,1.416331,1.268859,1.366209,1.298486,1.256066,1.244745,1.188220,1.152867
50%,1.595580,1.507767,1.444836,1.449998,1.075449,1.928563,1.077884,0.572921,0.998989,1.030186,...,1.056330,1.091333,1.715696,1.568368,1.799970,1.671493,1.587842,1.635624,1.553934,1.476619
75%,2.176113,2.031394,1.922228,1.616514,1.162879,3.474201,1.213967,1.652160,1.031505,1.078058,...,1.097205,1.158935,2.215145,1.902204,2.584207,2.347170,2.238324,2.258023,2.093279,1.977065
max,617438.722420,685771.750988,709612.451943,510001.000000,963001.000000,478001.000000,832001.000000,110001.000000,1.560646,1.927046,...,1.891079,2.176450,91.879279,21.928799,599645.128114,666008.905138,689162.554192,654717.981132,708164.265306,722917.666667


## log Ratio Disparity (LogRD)

In [8]:
new_cols = []


suffix = '_LogRD'

for i in ['CvdVax', 'CvdVaxAbove15', 'CvdVaxAbove20']:
    new_col = i+suffix
    T2[new_col] = np.log( (T2[ i + '_MWhiteRate'] + 0.0001) /( T2[i+'_MBlackRate']  + 0.0001) )
    new_cols.append(new_col)
    
    
for i in  ['FluVax', 'HighSchool', 'Bachelor', 'IT', 'Unempl', 'A15T74', 'A20T74',  'Above75', 'Above65', 'Above15', 'Above20',]:
    new_col = i+suffix
    T2[new_col] = np.log( (T2[ i + '_NHWhiteRate']+ 0.0001) / (T2[i+'_BlackRate']  + 0.0001) )
    new_cols.append(new_col)

    
for i in [ 'MeanInc', 'MedianInc']:
    new_col = i+suffix
    T2[new_col] = np.log((T2[ i + '_NHWhiteAvg'] + 0.0001)/ (T2[i+'_BlackAvg']  + 0.0001) )
    new_cols.append(new_col)

    
    
    

try:
    for i in ['FullCvdVax', 'FullCvdVaxAbove15', 'FullCvdVaxAbove20']:
        new_col = i+suffix
        T2[new_col] = np.log( (T2[ i + '_MWhiteRate'] + 0.0001)/( T2[i+'_MBlackRate']  + 0.0001) )
        new_cols.append(new_col)
except:
    print('No Full Cvd')
    
    
try:   
    for i in ['CvdVax', 'CvdVaxAbove15', 'CvdVaxAbove20']:
        new_col ='Z' + i+suffix
        T2[new_col] = np.log( (T2[ i + '_ZWhiteRate'] + 0.0001) /( T2[i+'_MBlackRate']  + 0.0001) )
        new_cols.append(new_col)
except:
    print('No ZCvd Info')
    
    
T2[new_cols].describe()

,CvdVax_LogRD,CvdVaxAbove15_LogRD,CvdVaxAbove20_LogRD,FluVax_LogRD,HighSchool_LogRD,Bachelor_LogRD,IT_LogRD,Unempl_LogRD,A15T74_LogRD,A20T74_LogRD,...,Above15_LogRD,Above20_LogRD,MeanInc_LogRD,MedianInc_LogRD,FullCvdVax_LogRD,FullCvdVaxAbove15_LogRD,FullCvdVaxAbove20_LogRD,ZCvdVax_LogRD,ZCvdVaxAbove15_LogRD,ZCvdVaxAbove20_LogRD
count,1175.000000,1175.000000,1175.000000,951.000000,1165.000000,1165.000000,1140.000000,1125.000000,1175.000000,1175.000000,...,1175.000000,1175.000000,1119.000000,824.000000,898.000000,898.000000,898.000000,1175.000000,1175.000000,1175.000000
mean,0.524233,0.472355,0.435466,0.693513,0.193988,1.995565,0.255920,1.500982,-0.008811,0.026815,...,0.056839,0.095397,0.636358,0.439201,0.758348,0.704271,0.665927,0.553103,0.496873,0.458441
std,0.716995,0.717809,0.713966,2.016048,1.140125,3.825898,1.456149,4.557620,0.085141,0.113957,...,0.083282,0.115248,0.532400,0.385305,1.179250,1.193273,1.198072,0.711808,0.713334,0.709855
min,-2.108302,-2.155716,-2.171438,-0.478035,-0.503350,-2.009909,-0.689154,-4.604972,-0.284224,-0.324852,...,-0.194033,-0.247408,-1.072308,-1.437234,-0.869928,-0.870160,-0.852872,-2.038950,-2.097386,-2.116721
25%,0.188359,0.148293,0.124766,0.245627,0.006276,0.256718,-0.045526,-0.995709,-0.036173,-0.014236,...,0.013101,0.036675,0.348069,0.238118,0.312040,0.261199,0.227984,0.218930,0.172457,0.142251
50%,0.467237,0.410630,0.367996,0.371562,0.072738,0.656775,0.075000,-0.557007,-0.001011,0.029740,...,0.054801,0.087400,0.539819,0.450035,0.587770,0.513717,0.462375,0.492024,0.440790,0.389755
75%,0.777540,0.708722,0.653485,0.480272,0.150898,1.245365,0.193893,0.502083,0.031019,0.075161,...,0.092766,0.147502,0.795318,0.643013,0.949416,0.853210,0.805718,0.814489,0.738726,0.681613
max,13.333335,13.438300,13.472474,13.142168,13.777810,13.077368,13.631589,11.608245,0.445100,0.655988,...,0.637147,0.777695,4.520476,3.087801,13.304093,13.409058,13.443232,13.391960,13.470431,13.491051


## Scaled Disparity (SD)

In [9]:
new_cols = []
suffix = '_SD'

# LeastOne CVD
for i in ['CvdVax', 'CvdVaxAbove15', 'CvdVaxAbove20']:
    popu_type = i.replace('CvdVax', '')
    popu_type = popu_type + '_' if len(popu_type) > 0 else popu_type
    
    new_col = i+ suffix
    AvgWBRate = (T2['CvdVax_ZWhiteNum'] + T2['CvdVax_BlackNum'])  / ( T2[popu_type + 'NHWhiteNum'] + T2[popu_type + 'BlackNum']) 
    T2[new_col] = T2[i + '_Disparity'] / (AvgWBRate * 100)
    new_cols.append(new_col)
    


# Rate Predictors
for i in  ['FluVax', 'HighSchool', 'Bachelor', 'IT', 'Unempl', 'A15T74', 'A20T74',  'Above75', 'Above65', 'Above15', 'Above20',]:
    new_col = i+suffix
    # T2[i + '_WhiteNum'] = T2[i + '_WhiteRate'] * T2['NHWhiteNum'] # Error here, keep here
    NHWhiteNum = T2[i + '_NHWhiteRate'] * T2['NHWhiteNum']
    BlackNum = T2[i + '_BlackRate'] * T2['BlackNum']
    AvgWBRate = (NHWhiteNum + BlackNum) / ( T2['NHWhiteNum'] + T2['BlackNum'])
    T2[new_col] = T2[i + '_Disparity'] / (AvgWBRate * 100)
    new_cols.append(new_col)


# Income Predictors
for i in [ 'MeanInc', 'MedianInc']:
    new_col = i+suffix
    T2[new_col] = T2[i+'_Disparity'] / T2[i+'_WholeAvg']
    new_cols.append(new_col)

    
    
# Full CVD
try:
    for i in ['FullCvdVax', 'FullCvdVaxAbove15', 'FullCvdVaxAbove20']:
        popu_type = i.replace('FullCvdVax', '')
        popu_type = popu_type + '_' if len(popu_type) > 0 else popu_type
        
        new_col = i+ suffix
        AvgWBRate = (T2['CvdVax_ZWhiteNum'] + T2['CvdVax_BlackNum'])  / ( T2[popu_type + 'NHWhiteNum'] + T2[popu_type + 'BlackNum'])
        T2[new_col] = T2[i + '_Disparity'] / (AvgWBRate * 100)
        new_cols.append(new_col)
except:
    print('No Full Cvd')
    
T2[new_cols].describe()

,CvdVax_SD,CvdVaxAbove15_SD,CvdVaxAbove20_SD,FluVax_SD,HighSchool_SD,Bachelor_SD,IT_SD,Unempl_SD,A15T74_SD,A20T74_SD,Above75_SD,Above65_SD,Above15_SD,Above20_SD,MeanInc_SD,MedianInc_SD,FullCvdVax_SD,FullCvdVaxAbove15_SD,FullCvdVaxAbove20_SD
count,1175.000000,1175.000000,1175.000000,951.000000,1165.000000,1165.000000,1140.000000,1123.000000,1175.000000,1175.000000,1175.000000,1175.000000,1175.000000,1175.000000,1119.000000,824.000000,898.000000,898.000000,898.000000
mean,0.317880,0.283517,0.259825,0.003219,0.000902,0.004055,0.000833,-0.010004,-0.000121,0.000209,0.005746,0.005167,0.000526,0.000862,0.469392,0.345519,0.325870,0.301376,0.283990
std,0.419419,0.430204,0.431581,0.002026,0.001620,0.006737,0.002334,0.024071,0.000868,0.001112,0.002926,0.002624,0.000773,0.001021,0.313480,0.314364,0.247750,0.247789,0.247674
min,-2.634281,-2.773383,-2.821180,-0.006101,-0.006524,-0.060939,-0.009878,-0.254746,-0.003170,-0.003618,-0.010974,-0.007381,-0.002134,-0.002795,-2.069020,-3.190452,-1.134394,-1.134842,-1.101688
25%,0.170875,0.138150,0.116569,0.002253,0.000063,0.002306,-0.000465,-0.013731,-0.000368,-0.000143,0.004029,0.003490,0.000130,0.000361,0.324125,0.220176,0.194027,0.170832,0.151327
50%,0.374586,0.334976,0.311068,0.003258,0.000710,0.005258,0.000734,-0.006449,-0.000010,0.000294,0.005982,0.005273,0.000538,0.000850,0.477959,0.394385,0.332511,0.294205,0.273378
75%,0.549333,0.515680,0.487017,0.004078,0.001424,0.007974,0.001818,0.004101,0.000307,0.000727,0.007834,0.007049,0.000898,0.001389,0.633311,0.532070,0.471498,0.445611,0.426312
max,1.200073,1.255981,1.267236,0.011852,0.010644,0.013323,0.010640,0.012574,0.003696,0.004833,0.014302,0.012163,0.004892,0.005571,2.079506,1.141091,0.993647,0.982698,0.991042


## Variable Construction

In [10]:
# How about other disparity. There unit is not meaningful. Maybe we can just leave them. 
# Absolute
T2['CvdVax_DisparityY'] = T2['CvdVax_Disparity']# *100
T2['CvdVaxAbove15_DisparityY'] = T2['CvdVaxAbove15_Disparity']# *100
T2['CvdVaxAbove20_DisparityY'] = T2['CvdVaxAbove20_Disparity']# *100
T2['FluVax_DisparityY'] = T2['FluVax_Disparity']# *100

# Ratio
T2['CvdVax_RDY'] = T2['CvdVax_RD']
T2['CvdVaxAbove15_RDY'] = T2['CvdVaxAbove15_RD']
T2['CvdVaxAbove20_RDY'] = T2['CvdVaxAbove20_RD']
T2['FluVax_RDY'] = T2['FluVax_RD']# *100



# logRatio
T2['CvdVax_LogRDY'] = T2['CvdVax_LogRD'] 
T2['CvdVaxAbove15_LogRDY'] = T2['CvdVaxAbove15_LogRD']
T2['CvdVaxAbove20_LogRDY'] = T2['CvdVaxAbove20_LogRD']
T2['FluVax_LogRDY'] = T2['FluVax_LogRD']



# Scaled 
T2['CvdVax_SDY'] = T2['CvdVax_SD']
T2['CvdVaxAbove15_SDY'] = T2['CvdVaxAbove15_SD']
T2['CvdVaxAbove20_SDY'] = T2['CvdVaxAbove20_SD']
T2['FluVax_SDY'] = T2['FluVax_SD']# *100




try:
    T2['FullCvdVax_DisparityY'] = T2['FullCvdVax_Disparity']# *100
    T2['FullCvdVaxAbove15_DisparityY'] = T2['FullCvdVaxAbove15_Disparity']# *100
    T2['FullCvdVaxAbove20_DisparityY'] = T2['FullCvdVaxAbove20_Disparity']# *100
    
    # Ratio
    T2['FullCvdVax_RDY'] = T2['FullCvdVax_RD']
    T2['FullCvdVaxAbove15_RDY'] = T2['FullCvdVaxAbove15_RD']
    T2['FullCvdVaxAbove20_RDY'] = T2['FullCvdVaxAbove20_RD']
    
    # Scaled 
    T2['FullCvdVax_SDY'] = T2['FullCvdVax_SD']
    T2['FullCvdVaxAbove15_SDY'] = T2['FullCvdVaxAbove15_SD']
    T2['FullCvdVaxAbove20_SDY'] = T2['FullCvdVaxAbove20_SD']
    
except:
    print('No Full Cvd')
    


In [11]:
T2['FacNum'] = T2[['FQHC', 'HOPD', 'PHMCY', 'RUHC']].sum(axis = 1)
T2['vehicle'] = 1- T2['EP_NOVEH']
T2['logFacNum' ] = np.log(T2['FacNum' ] + 1)
T2['logcases' ] = np.log(T2['cases' ] + 1)

T2['FacNumRate' ] = T2['FacNum' ]/T2['WholeNum']

T2['CaseRate' ] = T2['cases' ]/T2['WholeNum']


T2['Black_Prop' ] = T2['BlackNum' ]/T2['WholeNum']
T2['Segregation'] = T2['Segregation'] / 100
T2['log_WholeNum'] = np.log(T2['WholeNum'])
T2['org_WholeNum'] = T2['WholeNum'] + 0
T2['Selected'] = [1]*len(T2)



money_cols = ['MedianInc_WholeAvg', 'MedianInc_Disparity',]


for i in money_cols:
    T2[i] = T2[i] / 1000


rate1_cols = ['FacNumRate', 'CaseRate', 'republican_rate', 'vehicle',
             'Segregation','racial_weighted_bias','hesitancy','Black_Prop']

for i in rate1_cols:
    T2[i] = T2[i]  * 100
    
    
rate100_cols = ['HighSchool_WholeRate','HighSchool_Disparity', 
                 'IT_WholeRate','IT_Disparity',
                 'FluVax_WholeRate', 'FluVax_Disparity', 
                 'A15T74_WholeRate', 'A15T74_Disparity', 
                 'A20T74_WholeRate', 'A20T74_Disparity', 
                 'Above75_WholeRate', 'Above75_Disparity',
                 'Above65_WholeRate', 'Above65_Disparity',
                 'Above20_WholeRate', 'Above20_Disparity',
                 'Above15_WholeRate', 'Above15_Disparity']

    
display(HTML(T2[money_cols].describe().to_html()))
display(HTML(T2[rate1_cols].describe().to_html()))
display(HTML(T2[rate100_cols].describe().to_html()))

,MedianInc_WholeAvg,MedianInc_Disparity
count,1175.000000,824.000000
mean,54.966193,19.123584
std,14.166736,18.488456
min,26.738000,-190.604000
25%,46.389000,11.887750
50%,52.620000,20.650500
75%,60.150500,28.583000
max,142.299000,112.332000


,FacNumRate,CaseRate,republican_rate,vehicle,Segregation,racial_weighted_bias,hesitancy,Black_Prop
count,1175.000000,1175.000000,1175.000000,1175.000000,904.000000,1174.000000,1175.000000,1175.000000
mean,0.025074,9.809299,6401.116407,93.424596,45.763640,39.698312,18.194043,8.870473
std,0.016214,2.792497,1601.792408,4.250936,16.754712,1.774527,3.461612,12.283605
min,0.000000,2.323161,1124.920105,23.000000,0.040823,31.156179,7.000000,0.277816
25%,0.016450,8.013434,5423.985564,92.200000,33.346599,39.011379,17.000000,1.237518
50%,0.021573,9.865647,6724.937807,94.100000,46.656101,39.982319,18.000000,3.737138
75%,0.029048,11.511974,7603.072267,95.400000,57.470197,40.617693,20.000000,10.365744
max,0.158983,24.344356,9543.269231,100.000000,86.158768,44.452995,27.000000,76.973139


,HighSchool_WholeRate,HighSchool_Disparity,IT_WholeRate,IT_Disparity,FluVax_WholeRate,FluVax_Disparity,A15T74_WholeRate,A15T74_Disparity,A20T74_WholeRate,A20T74_Disparity,Above75_WholeRate,Above75_Disparity,Above65_WholeRate,Above65_Disparity,Above20_WholeRate,Above20_Disparity,Above15_WholeRate,Above15_Disparity
count,1175.000000,1165.000000,1175.000000,1140.000000,1174.000000,951.000000,1175.000000,1175.000000,1175.000000,1175.000000,1175.000000,1175.000000,1175.000000,1175.000000,1175.000000,1175.000000,1175.000000,1175.000000
mean,86.326979,8.074421,81.055745,6.592895,45.002555,14.503680,74.083869,-0.928942,67.787804,1.396610,8.254316,5.339689,19.546233,11.262454,76.042119,6.736299,82.338185,4.410747
std,5.940422,14.133217,7.043172,17.743925,8.972393,8.160478,2.310578,6.516359,2.524396,7.735750,2.024490,3.227954,4.330649,6.514739,3.257858,8.066036,2.693629,6.527773
min,55.500000,-36.900000,50.900000,-49.800000,10.000000,-22.000000,65.922204,-24.481251,57.211986,-26.597654,3.266964,-17.579909,8.719533,-13.417587,60.948260,-21.917808,69.658478,-17.636986
25%,83.100000,0.600000,77.050000,-3.900000,40.000000,10.000000,72.838772,-2.743063,66.496257,-0.991405,7.022232,3.395518,16.902640,6.897422,74.291770,2.822501,80.927254,1.101281
50%,87.700000,6.400000,81.900000,6.200000,47.000000,15.000000,74.112571,-0.075721,67.879450,2.038686,8.200936,5.275855,19.413296,10.960120,76.161768,6.684352,82.448082,4.532884
75%,90.600000,12.700000,85.900000,14.500000,51.000000,19.000000,75.289000,2.258810,69.223830,4.969221,9.423177,7.209802,21.881255,15.492765,78.085762,10.764705,84.035314,7.485500
max,98.900000,96.300000,97.000000,83.200000,65.000000,51.000000,85.659488,26.163522,77.912288,34.308096,17.853438,26.557623,37.536172,45.702306,87.625489,48.343816,90.998324,43.563941


In [12]:
print(T2['State'].unique())
print('State Number:', len(T2['State'].value_counts().sort_index()))

['California' 'Illinois' 'Indiana' 'Maine' 'New York' 'North Carolina'
 'Ohio' 'Oregon' 'Pennsylvania' 'South Carolina' 'Tennessee' 'Texas'
 'Virginia' 'West Virginia' 'Wisconsin']
State Number: 15


# Filtering Critieria

In [13]:
# first processing
critieria_nonan_list = ['CvdVax_DisparityY', 'FluVax_DisparityY']
critieria_nonan = {}
for col in critieria_nonan_list:
    critieria_nonan[col] = -T2[col].isna()
    
cirteria_within01_list = ['CvdVax_MWhiteRate', 'CvdVax_MBlackRate']
cirteria_within01 = {}
for col in cirteria_within01_list:
    cirteria_within01[col] = T2[col] <= 100


# Make Directory

In [14]:
from datetime import datetime
import os

folder = str(datetime.now()).split('.')[0].replace(':', '-').replace(' ', '_')
Result = 'StataReg/RegResult'
path = os.path.join(Result, folder)
print('Generate Path:', path)
os.makedirs(path)

Generate Path: StataReg/RegResult/2021-05-30_10-16-55


In [15]:
# T2[T2['CvdVax_BlackRate'] == 0]

# Filter Valid Counties

In [16]:
def prep_process(T2, cols, path):
    # cols: Y + Xs
    prefix_cols  = ['State', 'County']
    addtion_cols = [i for i in ['org_WholeNum', 'log_WholeNum', 'Black_Prop', 'BlackNum', 'NotInHighMobility', 'ReportNH'] if i not in cols]
    addtion_cols = addtion_cols if 'IT_Disparity' in cols else addtion_cols + ['IT_Disparity']
    
    # print('prefix cols:', prefix_cols)
    # print('addition cols:', addtion_cols)
    
    T2['Selected'] = 1
    print('Original Sample:', T2['Selected'].sum())
        
    selected_columns = prefix_cols + cols + addtion_cols
    
    
    
    ### Here
    critieria_nonan = {}
    critieria_nonan_list = selected_columns
    for col in critieria_nonan_list:
        # print(col)
        critieria_nonan[col] = (-T2[col].isna()).astype(int)
        
    for col in critieria_nonan:
        T2['Selected'] = T2['Selected']*critieria_nonan[col]
        print('The object after Dropping Nan:', col, T2['Selected'].sum())
        
    
    # cirteria_within01 = {}
    # cirteria_within01_list = ['CvdVax_MWhiteRate', 'CvdVax_MBlackRate']
    # for col in cirteria_within01_list:
    #     cirteria_within01[col] = (T2[col] < 100).astype(int)
    
    
    ### Here
    cirteria_others = {}
    cirteria_others['LargeBlackNum'] = (T2['BlackNum'] >= 273).astype(int)
    
    for col in cirteria_others:
        T2['Selected'] = T2['Selected']*cirteria_others[col]
        print('The object after the process of:', col, T2['Selected'].sum())
        
    
            
            
    pathT2 = os.path.join(path, 'T2.csv')
    print('Write T2 to:', pathT2 )
    T2.to_csv(pathT2)
    
    return T2, selected_columns


YXcols = ['CvdVax_DisparityY', 'HighSchool_Disparity', 'HighSchool_WholeRate']
T2, selected_columns = prep_process(T2, YXcols, path)
# T3 = T2[T2['Selected'] == 1]
# T3.shape

Original Sample: 1175
The object after Dropping Nan: State 1175
The object after Dropping Nan: County 1175
The object after Dropping Nan: CvdVax_DisparityY 1175
The object after Dropping Nan: HighSchool_Disparity 1165
The object after Dropping Nan: HighSchool_WholeRate 1165
The object after Dropping Nan: org_WholeNum 1165
The object after Dropping Nan: log_WholeNum 1165
The object after Dropping Nan: Black_Prop 1165
The object after Dropping Nan: BlackNum 1165
The object after Dropping Nan: NotInHighMobility 1164
The object after Dropping Nan: ReportNH 1164
The object after Dropping Nan: IT_Disparity 1135
The object after the process of: LargeBlackNum 891
Write T2 to: StataReg/RegResult/2021-05-30_10-16-55/T2.csv


In [17]:
T2

,State,County,Geographic Area Name,GEOID,ReportNH,Valid,WholeNum,WhiteNum,NHWhiteNum,BlackNum,...,FacNum,vehicle,logFacNum,logcases,FacNumRate,CaseRate,Black_Prop,log_WholeNum,org_WholeNum,Selected
0,California,Alameda,"Alameda, California",6001,1.0,1,1671329,823593,512134,184368,...,289,90.4,5.669881,11.389729,0.017292,5.289683,11.031221,14.329130,1671329,1
1,California,Alpine,"Alpine, California",6003,1.0,1,1129,767,692,4,...,0,94.0,0.000000,4.488636,0.000000,7.794508,0.354296,7.029088,1129,0
2,California,Amador,"Amador, California",6005,1.0,1,39752,35636,30742,1066,...,8,94.5,2.197225,8.207947,0.020125,9.229724,2.681626,10.590415,39752,1
3,California,Butte,"Butte, California",6007,1.0,1,219186,187742,155415,4158,...,39,92.8,3.688879,9.409109,0.017793,5.565136,1.897019,12.297676,219186,1
4,California,Calaveras,"Calaveras, California",6009,1.0,1,45905,41752,36672,498,...,5,97.5,1.791759,7.673688,0.010892,4.683586,1.084849,10.734329,45905,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1170,Wisconsin,Brown,"Brown, Wisconsin",55009,0.0,1,264542,232249,212441,7874,...,48,94.5,3.891820,10.473025,0.018145,13.361961,2.976465,12.485755,264542,1
1171,Wisconsin,Bayfield,"Bayfield, Wisconsin",55007,0.0,1,15036,12835,12661,94,...,5,96.7,1.791759,7.154615,0.033254,8.506252,0.625166,9.618203,15036,0
1172,Wisconsin,Barron,"Barron, Wisconsin",55005,0.0,1,45244,43096,42086,711,...,15,94.1,2.772589,8.700348,0.033154,13.270268,1.571479,10.719825,45244,1
1173,Wisconsin,Ashland,"Ashland, Wisconsin",55003,0.0,1,15562,13042,12790,83,...,8,90.6,2.197225,7.229114,0.051407,8.854903,0.533350,9.652587,15562,0


In [18]:

# msno.matrix(T2)

In [19]:
T3 = T2[T2['Selected'] == 1][selected_columns]
T3#.columns

,State,County,CvdVax_DisparityY,HighSchool_Disparity,HighSchool_WholeRate,org_WholeNum,log_WholeNum,Black_Prop,BlackNum,NotInHighMobility,ReportNH,IT_Disparity
0,California,Alameda,23.271405,5.7,88.4,1671329,14.329130,11.031221,184368,0.0,1.0,10.7
2,California,Amador,35.975046,27.2,90.0,39752,10.590415,2.681626,1066,1.0,1.0,34.5
3,California,Butte,16.405020,0.7,89.2,219186,12.297676,1.897019,4158,1.0,1.0,6.4
4,California,Calaveras,10.403638,24.9,90.2,45905,10.734329,1.084849,498,1.0,1.0,-13.8
5,California,Colusa,24.878446,18.7,71.3,21547,9.977992,1.308767,282,1.0,1.0,-0.6
...,...,...,...,...,...,...,...,...,...,...,...,...
1166,Wisconsin,Chippewa,32.920667,15.0,92.8,64658,11.076867,1.772402,1146,1.0,0.0,-13.5
1167,Wisconsin,Calumet,17.245062,25.5,94.2,50089,10.821557,0.848490,425,1.0,0.0,27.9
1170,Wisconsin,Brown,24.874032,12.5,92.1,264542,12.485755,2.976465,7874,1.0,0.0,6.1
1172,Wisconsin,Barron,15.024635,40.8,90.2,45244,10.719825,1.571479,711,1.0,0.0,43.6


In [20]:
# T3.dtype

In [21]:
pathDict = os.path.join(path, 'DataVarDict.csv')
pathStata = os.path.join(path, 'Data.dta')
    
T3.describe().T.to_csv(pathDict)
T3.to_stata(pathStata)

# Normalize and Generate DTA

In [22]:
def standardize_data(T2, YXcols, selected_columns, path):
    T3 = T2[T2['Selected'] == 1][selected_columns]
    
    
    pathDict = os.path.join(path, 'DataVarDict.csv')
    pathStata = os.path.join(path, 'Data.dta')
    
    T3.describe().T.to_csv(pathDict)
    T3.to_stata(pathStata)
    print('Write T3     to:', pathStata )
    
    # Do not normalize
    dummy_cols = ['republican', 'urban', 'ReportNH']
    # DependentVariables = [i for i in YXcols if i[-1] == 'Y']
    DependentVariables = [i for i in YXcols if 'DisparityY' in i or 'log' in i]
    not_normalized_cols = DependentVariables + dummy_cols
    
    T3Norm = T3.copy()
    for col in YXcols:
        if col in not_normalized_cols:
            continue
        df = T3Norm[col]
        T3Norm[col]=(df-df.mean())/df.std()
        
    pathNormDict = os.path.join(path, 'DataNormVarDict.csv')
    pathNormStata = os.path.join(path, 'DataNorm.dta')
    
    T3Norm.describe().T.to_csv(pathNormDict)
    T3Norm.to_stata(pathNormStata)
    print('Write T3Norm to:', pathNormStata )
        
    return T3, T3Norm


T3, T3Norm = standardize_data(T2, YXcols, selected_columns, path)
    

Write T3     to: StataReg/RegResult/2021-05-30_10-16-55/Data.dta
Write T3Norm to: StataReg/RegResult/2021-05-30_10-16-55/DataNorm.dta


# Stata Code

## Regression Code

In [23]:
def get_regression(Y, X, CV, addText, state_dummy_num = '16'):
    # Y: string
    # X: list
    # CV: dict(string: list)
    
    reg_string = ' '.join(['reg', Y] + X + sum([v for k, v in CV.items()], []))
    # reg_string = reg_string+'dummy
   
    
    reg_string = reg_string + ' state_dummy1-state_dummy{} '.format(state_dummy_num)
    if addText.get('W'):
        reg_string = reg_string + ' [pweight=org_WholeNum] '  ### 
    if addText.get('R'):
        reg_string = reg_string + ', r ' if ',' not in reg_string else reg_string + 'cluster(State) ' 
    if addText.get('C'):
        reg_string = reg_string + ', cluster(State) ' if ',' not in reg_string else reg_string + 'cluster(State)'
    # reg_string = reg_string + ', level()'
    
    return reg_string


## Outreg2 Code

In [24]:
def get_outreg2(Y, X, DocName, addText, replace = False):
    # Title = '_'.join(['Mdl', '.'.join([k for k, v in addText.items() if v] )])
    # Title = Title[:-1] if Title[-1] == '_' else Title
    # omit = "state_dummy1-state_dummy" + max_state + " o.state_dummy1-o.state_dummy" + max_state
    Title = Y
    TitleString = 'append ctitle({})'.format(Title) if not replace else 'replace ctitle({})'.format(Title)
    addTextString = 'addtext(SD, True, ' + ', '.join([k + ', ' + str(v) for k, v in addText.items()]) + ')' 
    d = ' '.join(['outreg2 using', 
                  DocName + ',', 
                  TitleString,  
                  addTextString, 
                  " keep (" +' '.join(X) + ') '
                  'excel',
                  "alpha(0.001, 0.01, 0.05, 0.10) symbol(***, **, *, +)"
                 ])
    return d
    

In [25]:
DataPath = os.path.join(os.getcwd(), path, 'Data.dta')
DataPath

'/Users/floydluo/Desktop/Covid19-Vaccination-Race-Disparity/StataReg/RegResult/2021-05-30_10-16-55/Data.dta'

 # Set Regression Variables

## Absolute Disparity

In [26]:
########### (1) Absolute Disparity ############


### TOCHANGE
# drop the large or not
# additonal_stata = ['drop if NotInHighMobility == 0'];  state_dummy_num = T3Norm['State'].nunique()
# additonal_stata = ['drop if Use_NoHisRace == 0']; T3Norm['State'].nunique() - len(NoHisRaceNum_States)

# use all
additonal_stata = []; state_dummy_num = T3Norm['State'].nunique()





Y  = 'FullCvdVax_DisparityY' #,'CvdVax_DisparityY' #, 'CvdVaxAbove20_DisparityY' #,'CvdVaxAbove15_DisparityY' #       'CvdVaxAbove15_DisparityY' # 'FullCvdVaxAbove20_DisparityY' #, 'FullCvdVaxAbove15_DisparityY' # ,   
Y2 = 'FluVax_DisparityY' 

X_list = [
# Base Model
[
'MedianInc_WholeAvg', 
'MedianInc_Disparity',

'HighSchool_WholeRate',
'HighSchool_Disparity',

'FacNumRate', # 'logFacNum', # 
'CaseRate', #'logcases', 

'IT_WholeRate',
'IT_Disparity',

'urban',

'vehicle',

'republican_rate', # 'republican', #  'republican_07', #  'republican_03',

'Segregation',

'racial_weighted_bias',

'hesitancy',

'Black_Prop',

],

# Robustness Check 

    
# [
# 'newcase_rate',
# ],
    
    
[
'FluVax_WholeRate',
'FluVax_Disparity'
],

[
'Above75_WholeRate',
'Above75_Disparity'
],

    
[
'A15T74_WholeRate',
'A15T74_Disparity'
],
    
    
# [
# 'Above15_WholeRate',
# 'Above15_Disparity'
# ],

# [
# 'Above20_WholeRate',
# 'Above20_Disparity'
# ],
]

YXcols = [Y, Y2] + sum(X_list, [])
# print(all_variables)
T2, selected_columns = prep_process(T2, YXcols, path)
T3, T3Norm = standardize_data(T2, YXcols, selected_columns, path)


Original Sample: 1175
The object after Dropping Nan: State 1175
The object after Dropping Nan: County 1175
The object after Dropping Nan: FullCvdVax_DisparityY 898
The object after Dropping Nan: FluVax_DisparityY 696
The object after Dropping Nan: MedianInc_WholeAvg 696
The object after Dropping Nan: MedianInc_Disparity 575
The object after Dropping Nan: HighSchool_WholeRate 575
The object after Dropping Nan: HighSchool_Disparity 575
The object after Dropping Nan: FacNumRate 575
The object after Dropping Nan: CaseRate 575
The object after Dropping Nan: IT_WholeRate 575
The object after Dropping Nan: IT_Disparity 575
The object after Dropping Nan: urban 575
The object after Dropping Nan: vehicle 575
The object after Dropping Nan: republican_rate 575
The object after Dropping Nan: Segregation 550
The object after Dropping Nan: racial_weighted_bias 550
The object after Dropping Nan: hesitancy 550
The object after Dropping Nan: Black_Prop 550
The object after Dropping Nan: FluVax_WholeRate

## Ratio Disparity

In [27]:
# ########### (2) Ratio Disparity ############


# ### TOCHANGE
# # drop the large or not
# # additonal_stata = ['drop if NotInHighMobility == 0'];  state_dummy_num = T3Norm['State'].nunique()
# # additonal_stata = ['drop if Use_NoHisRace == 0']; T3Norm['State'].nunique() - len(NoHisRaceNum_States)

# # use all
# additonal_stata = []; state_dummy_num = T3Norm['State'].nunique()




# ##### TOCHANGE
# Y  = 'CvdVax_RDY' #, 'CvdVaxAbove15_RDY' #      'CvdVaxAbove20_RDY' #,   'FullCvdVax_RDY' #,  'FullCvdVaxAbove20_RDY' #, 'FullCvdVaxAbove15_RDY' # 
# ######
# Y2 = 'FluVax_RDY' 

# X_list = [
# # Base Model
# [
# 'MedianInc_WholeAvg', 
# 'MedianInc_RD',

# 'HighSchool_WholeRate',
# 'HighSchool_RD',

# 'FacNumRate', # 'logFacNum', # 
# 'CaseRate', #'logcases', 

# 'IT_WholeRate',
# 'IT_RD',

# 'urban',

# 'vehicle',

# 'republican_rate', # 'republican', #  'republican_07', #  'republican_03',

# 'Segregation',

# 'racial_weighted_bias',

# 'hesitancy',

# 'Black_Prop',

# ],

# # Robustness Check 

# # [
# # 'newcase_rate',
# # ],
    
    
# [
# 'FluVax_WholeRate',
# 'FluVax_RD'
# ],

# [
# 'Above75_WholeRate',
# 'Above75_RD'
# ],

# [
# 'A15T74_WholeRate',
# 'A15T74_RD'
# ],
    
    
# # [
# # 'Above15_WholeRate',
# # 'Above15_RD'
# # ],
    
# # [
# # 'Above20_WholeRate',
# # 'Above20_RD'
# # ],

# ]

# YXcols = [Y, Y2] + sum(X_list, [])
# # print(all_variables)
# T2, selected_columns = prep_process(T2, YXcols, path)
# T3, T3Norm = standardize_data(T2, YXcols, selected_columns, path)


## Log Ratio Disparity

In [28]:
# ########### (2) Ratio Disparity ############


# ### TOCHANGE
# # drop the large or not
# # additonal_stata = ['drop if NotInHighMobility == 0'];  state_dummy_num = T3Norm['State'].nunique()
# # additonal_stata = ['drop if Use_NoHisRace == 0']; T3Norm['State'].nunique() - len(NoHisRaceNum_States)

# # use all
# additonal_stata = []; state_dummy_num = T3Norm['State'].nunique()




# ##### TOCHANGE
# Y  = 'CvdVax_LogRDY' #, 'CvdVaxAbove15_LogRDY' #      'CvdVaxAbove20_LogRDY' #,   'FullCvdVax_LogRDY' #,  'FullCvdVaxAbove20_LogRDY' #, 'FullCvdVaxAbove15_logRDY' # 
# ######
# Y2 = 'FluVax_LogRDY' 

# X_list = [
# # Base Model
# [
# 'MedianInc_WholeAvg', 
# 'MedianInc_LogRD',

# 'HighSchool_WholeRate',
# 'HighSchool_LogRD',

# 'FacNumRate', # 'logFacNum', # 
# 'CaseRate', #'logcases', 

# 'IT_WholeRate',
# 'IT_LogRD',

# 'urban',

# 'vehicle',

# 'republican_rate', # 'republican', #  'republican_07', #  'republican_03',

# 'Segregation',

# 'racial_weighted_bias',

# 'hesitancy',

# 'Black_Prop',

# ],

# # Robustness Check 

# # [
# # 'newcase_rate',
# # ],
    
    
# [
# 'FluVax_WholeRate',
# 'FluVax_LogRD'
# ],

# [
# 'Above75_WholeRate',
# 'Above75_LogRD'
# ],

# [
# 'A15T74_WholeRate',
# 'A15T74_LogRD'
# ],
    
    
# # [
# # 'Above15_WholeRate',
# # 'Above15_LogRD'
# # ],
    
# # [
# # 'Above20_WholeRate',
# # 'Above20_LogRD'
# # ],

# ]

# YXcols = [Y, Y2] + sum(X_list, [])
# # print(all_variables)
# T2, selected_columns = prep_process(T2, YXcols, path)
# T3, T3Norm = standardize_data(T2, YXcols, selected_columns, path)


## Scaled Disparity

In [29]:
# ########### (3) Scaled Disparity ############
# ### TOCHANGE
# # drop the large or not
# # additonal_stata = ['drop if NotInHighMobility == 0'];  state_dummy_num = T3Norm['State'].nunique()
# # additonal_stata = ['drop if Use_NoHisRace == 0']; T3Norm['State'].nunique() - len(NoHisRaceNum_States)

# # use all
# additonal_stata = []; state_dummy_num = T3Norm['State'].nunique()



# ######### TOCHANGE
# Y  =  'CvdVax_SDY' #,  'FullCvdVax_SDY' #, 'CvdVaxAbove20_SDY' #, 'CvdVaxAbove15_SDY' #     'FullCvdVaxAbove20_SDY' #, 'FullCvdVaxAbove15_SDY' #   
# #########



# Y2 = 'FluVax_SDY' 

# X_list = [
# # Base Model
# [
# 'MedianInc_WholeAvg', 
# 'MedianInc_SD',

# 'HighSchool_WholeRate',
# 'HighSchool_SD',

# 'FacNumRate', # 'logFacNum', # 
# 'CaseRate', #'logcases', 

# 'IT_WholeRate',
# 'IT_SD',

# 'urban',

# 'vehicle',

# 'republican_rate', # 'republican', #  'republican_07', #  'republican_03',

# 'Segregation',

# 'racial_weighted_bias',

# 'hesitancy',

# 'Black_Prop',

# ],

# # Robustness Check 

# # [
# # 'newcase_rate',
# # ],
    
# [
# 'FluVax_WholeRate',
# 'FluVax_SD'
# ],

# [
# 'Above75_WholeRate',
# 'Above75_SD'
# ],


# [
# 'A15T74_WholeRate',
# 'A15T74_SD'
# ],

    
# ]

# YXcols = [Y, Y2] + sum(X_list, [])
# # print(all_variables)
# T2, selected_columns = prep_process(T2, YXcols, path)
# T3, T3Norm = standardize_data(T2, YXcols, selected_columns, path)


## Final Data

In [30]:
T3.describe().T

,count,mean,std,min,25%,50%,75%,max
FullCvdVax_DisparityY,534.0,8.815191,7.759385,-11.219498,2.436799,8.388083,14.159413,5.360181e+01
FluVax_DisparityY,534.0,15.222846,6.217822,-12.000000,11.250000,16.000000,19.000000,3.600000e+01
MedianInc_WholeAvg,534.0,56.681934,14.733051,27.063000,47.394250,53.565500,61.361500,1.422990e+02
MedianInc_Disparity,534.0,20.185191,14.218214,-67.705000,13.753500,21.380000,28.364500,7.488700e+01
HighSchool_WholeRate,534.0,86.903558,5.217650,61.800000,83.900000,87.800000,90.875000,9.680000e+01
HighSchool_Disparity,534.0,6.859176,8.210362,-30.300000,2.200000,6.250000,11.300000,4.220000e+01
FacNumRate,534.0,0.023481,0.012618,0.000000,0.016192,0.020878,0.026878,9.809013e-02
CaseRate,534.0,9.635293,2.347065,2.323161,8.133694,9.640481,11.208254,1.840239e+01
IT_WholeRate,534.0,82.038577,6.963411,55.900000,78.100000,82.800000,86.900000,9.700000e+01
IT_Disparity,534.0,9.022846,11.141071,-21.900000,2.425000,8.400000,14.975000,7.230000e+01


In [31]:
# T3Norm
T3Norm.describe().T

,count,mean,std,min,25%,50%,75%,max
FullCvdVax_DisparityY,534.0,8.815191e+00,7.759385,-11.219498,2.436799,8.388083,14.159413,5.360181e+01
FluVax_DisparityY,534.0,1.522285e+01,6.217822,-12.000000,11.250000,16.000000,19.000000,3.600000e+01
MedianInc_WholeAvg,534.0,2.128967e-16,1.000000,-2.010373,-0.630398,-0.211527,0.317624,5.811224e+00
MedianInc_Disparity,534.0,4.823441e-17,1.000000,-6.181521,-0.452356,0.084034,0.575270,3.847305e+00
HighSchool_WholeRate,534.0,-1.077790e-15,1.000000,-4.811277,-0.575653,0.171810,0.761155,1.896724e+00
HighSchool_Disparity,534.0,-1.496930e-17,1.000000,-4.525888,-0.567475,-0.074196,0.540880,4.304417e+00
FacNumRate,534.0,-6.653022e-18,1.000000,-1.860826,-0.577616,-0.206239,0.269237,5.912797e+00
CaseRate,534.0,-9.646882e-16,1.000000,-3.115437,-0.639778,0.002210,0.670182,3.735347e+00
IT_WholeRate,534.0,-1.716480e-15,1.000000,-3.753703,-0.565610,0.109346,0.698138,2.148577e+00
IT_Disparity,534.0,-2.661209e-17,1.000000,-2.775572,-0.592209,-0.055905,0.534253,5.679630e+00


In [32]:
state_dummy_num = T3Norm['State'].nunique()
state_dummy_num

10

In [33]:
T3_Whole = T2[T2['Selected'] == 1]
# T3_Whole = T3_Whole[T3_Whole['NotInHighMobility'] == 1]
print(T3_Whole.shape)
T3_Whole['WholeNum'].sum()

(534, 286)


84959097

# Main Regression

In [34]:


def generate_main_regression_state_code(Y, Y2, X_list, additonal_stata = [], 
                                        convert_var = 'Black_Prop', 
                                        state_dummy_num = 16,
                                        winsor = 95, RESAMPLE_ROBUSTNESSCHECK = False,
                                       ):
    YXcols = [Y, Y2] + sum(X_list, [])
    
    # Normalized
    DataPath = os.path.join(os.getcwd(), path, 'DataNorm.dta')

    winsor_range = '(' + str(100- winsor) + ',' +  str(winsor) + ')'
    winsor_code = 'winsor2 {}, replace cuts ' + winsor_range
    # state_dummy_num = 16 
    
    
    
    L1 = [
        'clear all',
        'set more off',
        'use "' + DataPath + '"', '\n']
    
    L2 = [
        '\n', 
        # "preserve\n", 
        "tab State, generate(state_dummy)",
        '\n', ]
    

    L = L1  + additonal_stata + L2 + [winsor_code.format(i) for i in YXcols] + ['\n\n']


    addText = {
        'R': True,
        'C': True,
        'W': True,
    }
        
    if RESAMPLE_ROBUSTNESSCHECK == False:
        
        DocName = os.path.join(path, 'RegResult.doc')

        CV = {}

        # Title = 'FVAD' if 'flu' in Y.lower() else 'CVAD'
        

        Regression_Commands = []


        for idx, X in enumerate(X_list):
            print(state_dummy_num)
            X_used = sum(X_list[:idx+ 1], [])
            reg_string = get_regression(Y, X_used, CV, addText, state_dummy_num)

            Regression_Commands.append(reg_string) 

            replace = True if idx == 0 else False
            
            Title = Y.replace('Y', '').replace('Disparity', 'D')
            out = get_outreg2(Title, X_used, DocName, addText, replace)

            L = L +[reg_string+'\n', out, '\n']
            if X_used[-1] == convert_var:
                reg_string = get_regression(Y2, X_used, CV, addText, state_dummy_num)
                
                replace = True if idx == 0 else False
                Title = Y2.replace('Y', '').replace('Disparity', 'D')
                out = get_outreg2(Title, X_used, DocName, addText, False)
                L = L +[reg_string+'\n', out, '\n']

                Regression_Commands.append(reg_string) 


        visualization = []

        L = L + visualization

        stata_string =  '\n'.join(L)
        StataCodePath = os.path.join(os.getcwd(), path, 'do_file.do')
        print(StataCodePath, '\n\n\n')
        print(stata_string)
        with open(StataCodePath, 'w') as f:
            f.write(stata_string)
            
            
    elif RESAMPLE_ROBUSTNESSCHECK == True:
        DocName = os.path.join(path, 'RegResultRobust.doc')
        CV = {}
        idx = 0
        X_used = sum(X_list[:idx+ 1], [])
        reg_string = get_regression(Y, X_used, CV, addText, state_dummy_num)

        for idx in list(range(15)):
            replace = True if idx == 0 else False
            Title = Y.replace('Y', '').replace('Disparity', 'D')
            out = get_outreg2(Title, X_used, DocName, addText, replace)
            L = L + ['preserve\n', 'gsample 95, percent\n',  reg_string+'\n', out, '\n', 'restore\n\n\n']

        stata_string =  '\n'.join(L)
        StataCodePath = os.path.join(os.getcwd(), path, 'do_file_robust.do')
        print(StataCodePath, '\n\n\n')
        print(stata_string)
        with open(StataCodePath, 'w') as f:
            f.write(stata_string)
            
    return StataCodePath

            
    

if len(additonal_stata) != 0:
    print('++++++++++++++++')
    print('You add some additional stata code...')
    print(additonal_stata, state_dummy_num)
    print('++++++++++++++++')

convert_var = X_list[0][-1]
print(convert_var)
StataCodePath = generate_main_regression_state_code(Y, Y2, X_list,   
                                                    additonal_stata = additonal_stata,
                                                    # convert_var = 'Black_Prop', 
                                                    
                                                    convert_var =convert_var,
                                                    state_dummy_num = state_dummy_num,
                                                    winsor = 95, 
                                                    RESAMPLE_ROBUSTNESSCHECK = False)



Black_Prop
10
10
10
10
/Users/floydluo/Desktop/Covid19-Vaccination-Race-Disparity/StataReg/RegResult/2021-05-30_10-16-55/do_file.do 



clear all
set more off
use "/Users/floydluo/Desktop/Covid19-Vaccination-Race-Disparity/StataReg/RegResult/2021-05-30_10-16-55/DataNorm.dta"




tab State, generate(state_dummy)


winsor2 FullCvdVax_DisparityY, replace cuts (5,95)
winsor2 FluVax_DisparityY, replace cuts (5,95)
winsor2 MedianInc_WholeAvg, replace cuts (5,95)
winsor2 MedianInc_Disparity, replace cuts (5,95)
winsor2 HighSchool_WholeRate, replace cuts (5,95)
winsor2 HighSchool_Disparity, replace cuts (5,95)
winsor2 FacNumRate, replace cuts (5,95)
winsor2 CaseRate, replace cuts (5,95)
winsor2 IT_WholeRate, replace cuts (5,95)
winsor2 IT_Disparity, replace cuts (5,95)
winsor2 urban, replace cuts (5,95)
winsor2 vehicle, replace cuts (5,95)
winsor2 republican_rate, replace cuts (5,95)
winsor2 Segregation, replace cuts (5,95)
winsor2 racial_weighted_bias, replace cuts (5,95)
winsor2 hesitancy, r

# Run Regression

In [35]:
import subprocess

## Do some processing in Python
## Set do-file information
dofile = StataCodePath

stata_app_path = '/Applications/Stata/StataSE.app/Contents/MacOS/StataSE'
cmd = [stata_app_path, "do", dofile, "mpg", "weight", "foreign"]

## Run do-file
subprocess.call(cmd) 